To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [3]:
#@title Colab Extra Install { display-mode: "form" }
# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


### Unsloth

Load up `Qwen 2.5 3B Instruct`, and set parameters

In [5]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/dev/shm/qwen25_3B/",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.1. vLLM: 0.8.2.
   \\   /|    NVIDIA H20. Num GPUs = 1. Max memory: 95.1 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /dev/shm/qwen25_3B/ with actual GPU utilization = 49.79%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 95.1 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 226.
Unsloth: vLLM's KV Cache can use up to 41.38 GB. Also swap space = 6 GB.
INFO 03-27 00:16:53 [config.py:585] This model supports multiple tasks: {'classify', 'generate', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 03-27 00:16:53 [arg_utils.py:1854] --quantization bitsand

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-27 00:16:55 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-27 00:16:56 [model_runner.py:1146] Model loading took 2.1585 GB and 1.688970 seconds
INFO 03-27 00:17:00 [worker.py:267] Memory profiling takes 3.61 seconds
INFO 03-27 00:17:00 [worker.py:267] the current vLLM instance can use total_gpu_memory (95.10GiB) x gpu_memory_utilization (0.50) = 47.35GiB
INFO 03-27 00:17:00 [worker.py:267] model weights take 2.16GiB; non_torch_memory takes 0.12GiB; PyTorch activation peak memory takes 1.26GiB; the rest of the memory reserved for KV Cache is 43.81GiB.
INFO 03-27 00:17:00 [executor_base.py:111] # cuda blocks: 79747, # CPU blocks: 10922
INFO 03-27 00:17:00 [executor_base.py:116] Maximum concurrency for 1024 tokens per request: 1246.05x
INFO 03-27 00:17:05 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.

Capturing CUDA graph shapes: 100%|██████████| 32/32 [00:34<00:00,  1.08s/it]

INFO 03-27 00:17:39 [model_runner.py:1570] Graph capturing finished in 35 secs, took 0.79 GiB
INFO 03-27 00:17:39 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 43.53 seconds



Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.3.19 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [6]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('modelscope/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

The repository for gsm8k contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/gsm8k.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [9]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 8,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 2 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [10]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 119,734,272/3,000,000,000 (3.99% trained)


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
<reasoning>
To find the total number of coins in each chest, we need to calculate the coins allocated to the chests for gold, silver, and bronze separately, and then divide these totals equally among the 5 chests. 

1. For the gold coins: The pirate has 3500 gold coins to be distributed equally. Thus, the gold coins in each chest are:
   \(\frac{3500}{5} = 700\)

2. For the silver coins: The pirate has 500 silver coins to be distributed equally across the 5 chests:
   \(\frac{500}{5} = 100\)

3. For the bronze coins: The pirate has twice as many bronze coins as silver coins. So, he has \(2 \times 500 = 1000\) bronze coins. These are distributed equally across the 5 chests:
   \(\frac{10 
Extracted:
<reasoning>
To find the total number of coins in each chest, we need to calc

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
8,0.000000,0.048547,0.093797,198.406250,0.000508,0.048547,0.000000,0.000000,0.000000,0.000000
16,0.000100,0.034672,0.116113,199.125000,0.002942,0.034672,0.000000,0.000000,0.000000,0.000000
24,0.000000,-0.005719,0.149764,193.218750,0.000655,-0.005719,0.000000,0.000000,0.000000,0.000000
32,0.000000,-0.034750,0.171687,183.656250,0.000987,-0.034750,0.000000,0.000000,0.000000,0.000000
40,0.000100,0.061859,0.127265,197.187500,0.001275,0.061859,0.000000,0.000000,0.000000,0.000000
48,0.000100,0.073547,0.081194,198.656250,0.002565,0.073547,0.000000,0.000000,0.000000,0.000000
56,0.000200,0.035234,0.109740,196.812500,0.005801,0.035234,0.000000,0.000000,0.000000,0.000000
64,0.000300,0.099875,0.031687,193.718750,0.006879,0.099875,0.000000,0.000000,0.000000,0.000000
72,0.000200,0.058141,0.090740,199.437500,0.006077,0.058141,0.000000,0.000000,0.000000,0.000000
80,0.000300,0.103641,0.032599,192.078125,0.007501,0.103641,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
<reasoning>
To determine how much more is the monthly payment on the house compared to the trailer, we need to calculate the monthly payment for each, and then find the difference. 

The monthly payment can be calculated using the formula for a fixed monthly installment on a loan:
\[ \text{Monthly Payment} = \frac{P \times r}{1 - (1 + r)^{-n}} \]
where:
- \( P \) is the principal loan amount.
- \( r \) is the monthly interest rate (annual interest rate divided by 12).
- \( n \) is the total number of payments (years times 12).

Since the interest rate isn't specified, we'll assume both the house and the trailer loan will have the same interest rate for simplicity, say \( r \).

First

TrainOutput(global_step=250, training_loss=0.0003581883055740036, metrics={'train_runtime': 1465.2736, 'train_samples_per_second': 1.365, 'train_steps_per_second': 0.171, 'total_flos': 0.0, 'train_loss': 0.0003581883055740036})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [16]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "请给我写一首歌词，有浅浅的悲伤感，但整体风格比较平静的"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.82s/it, est. speed input: 17.01 toks/s, output: 131.10 toks/s]


'【浅浅的悲伤】\n\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌\n\n（第一节）\n曾经那些微小的幸福\n如花儿般悄然绽放\n然而那些时光的流转\n却又在瞬间褪去\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌\n\n（第二节）\n回忆起那片熟悉的天空\n但如今它已不再\n梦里重逢的笑脸\n却在梦后带走了笑容\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌\n\n（尾声）\n那年的笑容\n已化作了月亮\n在每个午夜的长夜\n守护着那些寂静的梦\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌\n\n（桥段）\n可是即使在暗夜\n那光芒从未褪去\n温柔的月光\n会照亮每一个寻找的路\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌\n\n（终曲）\n那深藏的忧伤\n在月光中轻轻回响\n在平凡的日子里\n温柔地寻找着遗失的温柔\n（副歌）\n在那平凡的日子里\n寻找着遗失的温柔\n那深藏的忧伤\n如同深夜的月\n悄悄地，静静地在心中流淌'

In [17]:
print (output)

【浅浅的悲伤】

（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌

（第一节）
曾经那些微小的幸福
如花儿般悄然绽放
然而那些时光的流转
却又在瞬间褪去
（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌

（第二节）
回忆起那片熟悉的天空
但如今它已不再
梦里重逢的笑脸
却在梦后带走了笑容
（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌

（尾声）
那年的笑容
已化作了月亮
在每个午夜的长夜
守护着那些寂静的梦
（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌

（桥段）
可是即使在暗夜
那光芒从未褪去
温柔的月光
会照亮每一个寻找的路
（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌

（终曲）
那深藏的忧伤
在月光中轻轻回响
在平凡的日子里
温柔地寻找着遗失的温柔
（副歌）
在那平凡的日子里
寻找着遗失的温柔
那深藏的忧伤
如同深夜的月
悄悄地，静静地在心中流淌


And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.06s/it, est. speed input: 14.05 toks/s, output: 29.09 toks/s]


'<reasoning>\nTo find out how many times the letter \'r\' appears in the word "strawberry", we can go through the word character by character and count each occurrence of \'r\'. In "strawberry", the letter \'r\' appears 3 times: once in the beginning, once in the middle, and once at the end of the word.\n</reasoning>\n<answer>\n3\n</answer>'

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
